In [2]:
import uuid

from confluent_kafka import Producer, Consumer


p = Producer({
    'bootstrap.servers': 'pkc-ew3qg.asia-southeast2.gcp.confluent.cloud:9092',
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': 'DVWJBMS4P4T2HOZJ',
    'sasl.password': 'QqkSBX2unE+jGW8o4CvmC75bvdeGg5q8xqNB645oHAfx6zdSNUQRrqR40aEByvmW'
})


def acked(err, msg):
    """Delivery report callback called (from flush()) on successful or failed delivery of the message."""
    if err is not None:
        print("failed to deliver message: {}".format(err.str()))
    else:
        print("produced to: {} [{}] @ {}".format(msg.topic(), msg.partition(), msg.offset()))


p.produce('test-topic', value='python test value', callback=acked)

# flush() is typically called when the producer is done sending messages to wait
# for outstanding messages to be transmitted to the broker and delivery report
# callbacks to get called. For continous producing you should call p.poll(0)
# after each produce() call to trigger delivery report callbacks.
p.flush(10)

c = Consumer({
    'bootstrap.servers': 'pkc-ew3qg.asia-southeast2.gcp.confluent.cloud:9092',
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': 'DVWJBMS4P4T2HOZJ',
    'sasl.password': 'QqkSBX2unE+jGW8o4CvmC75bvdeGg5q8xqNB645oHAfx6zdSNUQRrqR40aEByvmW',
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest'
})

c.subscribe(['test-topic'])

try:
    while True:
        msg = c.poll(0.1)  # Wait for message or event/error
        if msg is None:
            # No message available within timeout.
            # Initial message consumption may take up to `session.timeout.ms` for
            #   the group to rebalance and start consuming.
            continue
        if msg.error():
            # Errors are typically temporary, print error and continue.
            print("Consumer error: {}".format(msg.error()))
            continue

        print('consumed: {}'.format(msg.value()))

except KeyboardInterrupt:
    pass

finally:
    # Leave group and commit final offsets
    c.close()

produced to: test-topic [0] @ 2
consumed: b'hello kafka'
consumed: b'hello kafka'
consumed: b'python test value'
consumed: b'hello kafka'
consumed: b'hello kafka'
consumed: b'hello kafka'
consumed: b'python test value'
consumed: b'hello kafka'
consumed: b'hello kafka'
